In [ ]:
# pip install ipython-sql

%load_ext sql
%sql postgresql://###

In [ ]:
df= %sql select user_emails.user_id, user_emails.email, gmail_accounts.refresh_token from user_emails inner join gmail_accounts on user_emails.user_id = gmail_accounts.user_id where gmail_accounts.refresh_token <> 'None'
df = df.DataFrame()

In [ ]:
from oauth2client.client import GoogleCredentials
from apiclient import discovery
import pandas as pd
import json
import re

In [ ]:
SCOPES = 'https://www.googleapis.com/auth/gmail.readonly'
CLIENT_SECRET_FILE = '###.json'
APPLICATION_NAME = 'Gmail API Python Quickstart'
CLIENT_ID = ###
CLIENT_SECRET = '###'
TOKEN_URI = 'https://accounts.google.com/o/oauth2/token'
USER_AGENT = '###'


DomainSubjectCounts = {}
n = 0

for idx, (User_ID, _, REFRESH_TOKEN) in df.iterrows():

    try:
        credentials = GoogleCredentials(None, CLIENT_ID, CLIENT_SECRET, REFRESH_TOKEN, None, TOKEN_URI, USER_AGENT)
        service = discovery.build('gmail', 'v1', credentials=credentials)
        user_messages = service.users().messages().list(userId='me').execute()
        
        
        for message in user_messages.get('messages', []):
            
            msg = service.users().messages().get(userId='me', id=message['id'], format='metadata').execute()
            
            FromSubject = {}
            for header in msg['payload']['headers']:
                if (header['name'] == 'From') | (header['name'] == 'Subject'):
                    FromSubject[header['name']] = re.sub(r'[^\x00-\x7f]', r' ', header['value']).lower()
                    if len(FromSubject) > 1:
                        break
            
            new_domain = FromSubject['From']
            new_subject = FromSubject['Subject']
            
            
            if new_domain in DomainSubjectCounts:
                if new_subject in DomainSubjectCounts[new_domain]:
                    DomainSubjectCounts[new_domain][new_subject] += 1
                else:
                    DomainSubjectCounts[new_domain][new_subject] = 1
            else:
                DomainSubjectCounts[new_domain] = {new_subject : 1}
            
                
        n += 1
        print('{}/{} Users: Analysed!'.format(n, idx+1))
        
        
        if not len(DomainSubjectCounts) % 500:
            with open(r'.\Desktop\DomainSubjectCounts.txt', 'w') as txtFile:
                json.dump(DomainSubjectCounts, txtFile)
                
    except:
        continue